Your research group is looking at the affects of covid on government spending. You've found a kaggle dataset that has collected Covid-19 research papers, and the group would like to use the number of papers published by country as an indicator of the level of funded research. The group has decided to look for country name in the title and to map author emails to country (via a separate process). So you will need to store this metadata in a database structure.

You are provided with a small subset of the CORD19 dataset in a zipfile. 

Design a data schema to hold the necessary data, and write a python function that will upload them into your table structure. Use the sqlite in memory database (syntax provided in the code blocks).

helpers

https://github.com/allenai/cord19

official

https://www.kaggle.com/datasets/allen-institute-for-ai/CORD-19-research-challenge?select=metadata.readme

In [1]:
import json
import os
import pandas as pd
import sqlite3
from zipfile import ZipFile

"""
NOTES
- Two separate tables are formed here, papers (paper_id, paper_title, publish_country) & authors (paper_id, auth_email)
- The process of standardizing the extraction of countries requires a more sophisticated processes leveraging a list of all possible countries mapped back to 
ISO 3166-1 standard country list found here https://www.iso.org/obp/ui/#search
- column headings have been changed to reflect the data with more accuracy when storing,
eg paper_title instead of title to correctly reference the title of the article without any ambiguity
eg auth_email instead of email to correctly represent the author
eg publish_country to disambiguate the reference from other possibilities such as author's country, location of disease etc
- The separate task of mapping authors' emails to a country will also require a more sophisticated process, perhaps looking at the domain of the email server 
(if it belongs to an educational institution) and mapping to the home country of that institution.
"""
    
def create_database():
    conn = sqlite3.connect(":memory:")
    c = conn.cursor()

    # Create 2 separate tables for "papers" & "authors"
    c.execute("""
        CREATE TABLE IF NOT EXISTS papers (
            paper_id TEXT PRIMARY KEY,
            paper_title TEXT,
            publish_country TEXT
        )
    """)

    c.execute("""
        CREATE TABLE IF NOT EXISTS authors (
            paper_id TEXT,
            auth_email TEXT,
            FOREIGN KEY (paper_id) REFERENCES papers (paper_id)
        )
    """)

    conn.commit()
    return conn

def load_cord19_files(filename):
    # Create database
    conn = create_database()
    c = conn.cursor()

    # Open files from zip
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('cord19_mini')

    # Cycle through the files & locate json structures
    for root, dirs, files in os.walk('cord19_mini'):
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(root, file), 'r') as json_file:
                    data = json.load(json_file)

                    # Extract information
                    paper_id = data['paper_id']
                    paper_title = data['metadata']['title']
                    publish_country = extract_country_from_title(paper_title)

                    # Insert into the 'papers' table: paper_id, paper_title, publish_country
                    c.execute("INSERT INTO papers VALUES (?, ?, ?)", (paper_id, paper_title, publish_country))

                    # Insert into the 'author' table: paper_id, auth_email
                    for author in data['metadata']['authors']:
                        auth_email = author.get('email', None)
                        if auth_email:
                            c.execute("INSERT INTO authors VALUES (?, ?)", (paper_id, auth_email))

    conn.commit()
    return conn
    
"""
Given the non standard way countries may be reference in free text and articles, the title would need a fuzzy match search leveraging a function like extract_country_from_title (below).
You could start by creating coutnries = ISO 3166 Standard found at https://www.iso.org/obp/ui/#search. 
"""
def extract_country_from_title(paper_title):
    # Extract country from the paper_title
    countries = ["insert list of comma separated countries here"]  # Sample list of countries
    for publish_country in countries:
        if publish_country in paper_title:
            return publish_country
    return None


if __name__ == '__main__':
    

#     ## Syntax for using sqlite3
#     conn = sqlite3.connect(":memory:")
#     c=conn.cursor()
#     c.execute("CREATE TABLE foo (bar_one text, bar_two text)")
#     bars = [('a','b')]
#     c.executemany("INSERT INTO foo VALUES (?, ?)", bars)
#     conn.commit()
#     fptr = open(os.environ['OUTPUT_PATH'], 'w')  # file writing
#     conn.close()
#     ## End Synatx

    filename = 'cord19_mini.zip'
    load_cord19_files(filename)


NameError: name 'zipfile' is not defined